In [1]:
import numpy as np
import pandas as pd
import gensim
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rosiezhao/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
df = pd.read_csv('../data/data.csv')

/Users/rosiezhao/aiml/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
breitbart = df.loc[df['publication']=='Breitbart']

In [4]:
breitbart

,id,title,author,date,content,year,month,publication,category,digital,section,url
18049,26539,CNN's Zeleny: 'Hard to Imagine' Obama Would Ha...,Ian Hanchett,2017-01-17,MOST POPULAR[On Tuesday’s broadcast of CNN’s “...,2017.0,1.0,Breitbart,general,1.0,NaN,NaN
18050,26540,American Students on Spring Break Chant 'Build...,Katherine Rodriguez,2017-03-21,A group of American spring break revelers repo...,2017.0,3.0,Breitbart,general,1.0,NaN,NaN
18051,26541,Surge in 'Honour Crimes' and Forced Marriages ...,Liam Deacon,2017-04-08,So-called “honour crimes” have risen by 40 per...,2017.0,4.0,Breitbart,general,1.0,NaN,NaN
18052,26542,MILO Announces New Media Venture - Breitbart,Lucas Nolan,2017-04-28,Former Breitbart Senior Editor MILO has announ...,2017.0,4.0,Breitbart,general,1.0,NaN,NaN
18053,26543,Jared Kushner at Center of Media Spotlight on ...,Penny Starr,2017-05-27,The focus of the continuous media reports of a...,2017.0,5.0,Breitbart,general,1.0,NaN,NaN
18054,26544,Watch: The Resistance Creates Anti-Trump Aerob...,Amanda House,2017-05-13,Two California women have choreographed an ant...,2017.0,5.0,Breitbart,general,1.0,NaN,NaN
18055,26545,Watch: Celebrities Beg Congress to 'Obstruct' ...,Jerome Hudson,2017-01-03,A group of Hollywood celebrities is urging Ame...,2017.0,1.0,Breitbart,general,1.0,NaN,NaN
18056,26546,20-Year-Old Brit Dies Fighting Islamic State i...,Nick Hallett,2017-01-03,A 20-year-old British man who traveled to Syri...,2017.0,1.0,Breitbart,general,1.0,NaN,NaN
18057,26547,"Meet Judith Clark, the Domestic Terrorist Gran...",Daniel J. Flynn,2017-01-17,“Were you on drugs?” New York Governor Andrew ...,2017.0,1.0,Breitbart,general,1.0,NaN,NaN
18058,26548,Drug Stocks Drop as Trump Backs Price Bidding,Dr. Susan Berry,2017-01-11,Drug stocks plummeted as President-Elect Donal...,2017.0,1.0,Breitbart,general,1.0,NaN,NaN


In [5]:
titles = breitbart['title'].fillna('').astype(str).tolist()#.str.replace('[^a-zA-Z]',' ').str.lower()
articles = breitbart['content'].fillna('').astype(str).tolist()#.str.replace('[^a-zA-Z]',' ').str.lower()#maybe?

In [6]:
"""
Remove NaN values from the articles and tokenize
"""
cleaned_articles = [x.split() for x in articles if str(x) != 'nan']
cleaned_titles = [x.split() for x in titles if str(x) != 'nan']
print("len(cleaned_articles) = ", len(cleaned_articles))
print("len(cleaned_titles) = ", len(cleaned_titles))
print(cleaned_articles[0])
print(cleaned_titles[0])

len(cleaned_articles) =  23791
len(cleaned_titles) =  23791
['MOST', 'POPULAR[On', 'Tuesday’s', 'broadcast', 'of', 'CNN’s', '“Situation', 'Room,”', 'CNN', 'Senior', 'Washington', 'Correspondent', 'Jeff', 'Zeleny', 'stated', 'that', 'Chelsea', 'Manning’s', 'transition', 'from', 'a', 'man', 'to', 'a', 'woman', '“certainly', 'played', 'into”', 'President', 'Obama’s', 'decision', 'to', 'commute', 'Manning’s', 'sentence,', 'and', '“Without', 'that,', 'it’s', 'hard', 'to', 'imagine,', 'I', 'think,', 'this', 'president', 'would', 'have', 'done', 'that.”,', 'advertisement,', 'Zeleny', 'said,', '“I', 'think', 'a', 'question', 'that', 'this', 'president', 'the', 'White', 'House', 'will', 'have', 'to', 'answer', 'here', '—', 'will', 'answer,', 'and', 'I', 'think', 'it’s', 'an', 'important', 'one,', 'if', '—', 'how', 'much', 'was', 'the', 'personal', 'story', 'of', 'Chelsea', 'Manning', 'involved', 'in', 'this,', 'because', 'the', 'outcry', 'from', 'the', 'left', 'was', 'so', 'strong', 'on', 'this

In [7]:
"""
Input preprocessed articles through the phraser to form bigrams
"""
phrases = gensim.models.phrases.Phrases(cleaned_articles)
phraser = gensim.models.phrases.Phraser(phrases)
train_phrased = phraser[cleaned_articles]

title_phrases = gensim.models.phrases.Phrases(cleaned_titles)
title_phraser = gensim.models.phrases.Phraser(title_phrases)
title_train_phrased = title_phraser[cleaned_titles]

In [8]:
climate_seed=['emission', 'greenhouse_gas', 'ice_sheet', 'glacier', 'climate', 'temperature_rising', 'earth_warming', 'planet_warming', 'climate_change', 'ecologically', 'dioxide', 'human_activity', 'heat_trapping', 'global_warming', 'carbon_dioxide', 'sea_level', 'west_antarctic', 'scientific_consensus', 'average_temperature', 'human_induced', 'methane_emission', 'warming_planet', 'methane', 'environmentally', 'atmosphere', 'carbon_emission', 'temperature_rise', 'denier', 'environment', 'reduce_emission', 'warming_temperature', 'ocean_temperature', 'climate_changing', 'ecosystem']

In [10]:
"""
Re-creating the article contents by joining the tokenized words
with the newly added bigrams
"""
cleaned_articles = []
cleaned_titles = []

for i in range(len(train_phrased)):
  cleaned_articles.append(" ".join(train_phrased[i]))
  cleaned_titles.append(" ".join(title_train_phrased[i]))
  ##

  
print(cleaned_articles[0])
print(cleaned_titles[0])

MOST_POPULAR[On Tuesday’s_broadcast of CNN’s_“Situation Room,” CNN_Senior Washington Correspondent Jeff_Zeleny stated_that Chelsea Manning’s transition from a man to a woman “certainly played into” President_Obama’s decision to commute Manning’s sentence, and “Without that, it’s_hard to imagine, I_think, this president would have_done that.”, advertisement, Zeleny said,_“I think a question that this president the White_House will have to answer here — will answer, and I_think it’s an_important one, if — how_much was the personal story of Chelsea_Manning involved_in this, because the outcry_from the left was so strong on this._And she’s having a difficult time in federal_prison, no question. But, to me, that is a central question here. Without that, you have to wonder_if the outcome would_be the same. I_think it might not be.”, He_added, “[B]ecause she transitioned_from a man to a woman, I_think all of that certainly played into this. Without that, it’s_hard to imagine, I_think, this pr

In [11]:
"""
Creating dictionary of word frequencies and corpus for TF-IDF
"""
mydict = gensim.corpora.Dictionary([gensim.utils.simple_preprocess(line) for line in cleaned_articles])
corpus = [mydict.doc2bow(gensim.utils.simple_preprocess(line)) for line in cleaned_articles]

# mydict = gensim.corpora.Dictionary([gensim.utils.simple_preprocess(line) for line in cleaned_titles])
# corpus = [mydict.doc2bow(gensim.utils.simple_preprocess(line)) for line in cleaned_titles]

In [12]:
"""
Viewing first 10 articles with each word and its frequency in the article
"""
i = 0

for doc in corpus:
    print([[mydict[id], freq] for id, freq in doc])
    i = i+1
    if i == 10:
      break

[['advertisement', 1], ['all', 1], ['an_important', 1], ['and', 2], ['answer', 2], ['be', 1], ['because', 1], ['but', 1], ['central', 1], ['certainly', 2], ['chelsea', 1], ['chelsea_manning', 1], ['cnn', 1], ['cnn_senior', 1], ['commute', 1], ['correspondent', 1], ['decision', 1], ['difficult', 1], ['ecause', 1], ['federal_prison', 1], ['from', 1], ['have', 2], ['have_done', 2], ['having', 1], ['he_added', 1], ['here', 2], ['how_much', 1], ['i_think', 5], ['ian_hanchett', 1], ['ianhanchett', 1], ['if', 1], ['imagine', 2], ['in', 1], ['into', 2], ['involved_in', 1], ['is', 1], ['it', 4], ['jeff_zeleny', 1], ['left', 1], ['man', 2], ['manning', 2], ['me', 1], ['mediaite', 1], ['might', 1], ['most_popular', 1], ['no', 1], ['not', 1], ['of', 3], ['on', 2], ['on_twitter', 1], ['one', 1], ['outcome', 1], ['outcry_from', 1], ['personal', 1], ['played', 2], ['president', 3], ['president_obama', 1], ['question', 3], ['room', 1], ['s_', 1], ['s_broadcast', 1], ['s_hard', 2], ['said', 1], ['same'

In [13]:
"""
Generating the TF-IDF model
"""
tfidf = gensim.models.TfidfModel(corpus, smartirs='ntc')

In [14]:
"""
Viewing first 10 words in the tfidf model and their frequency
"""
i = 0

for doc in tfidf[corpus]:
    print([[mydict[id], np.around(freq, decimals=5)] for id, freq in doc])
    i = i+1
    if i == 10:
      break

[['advertisement', 0.00271], ['all', 0.0191], ['an_important', 0.10471], ['and', 0.00149], ['answer', 0.15163], ['be', 0.01651], ['because', 0.0276], ['but', 0.01251], ['central', 0.08689], ['certainly', 0.15253], ['chelsea', 0.12643], ['chelsea_manning', 0.17338], ['cnn', 0.06623], ['cnn_senior', 0.18756], ['commute', 0.16966], ['correspondent', 0.11288], ['decision', 0.05765], ['difficult', 0.08196], ['ecause', 0.20428], ['federal_prison', 0.14474], ['from', 0.00991], ['have', 0.02028], ['have_done', 0.19725], ['having', 0.05353], ['he_added', 0.0652], ['here', 0.08069], ['how_much', 0.09815], ['i_think', 0.23376], ['ian_hanchett', 0.13106], ['ianhanchett', 0.08696], ['if', 0.02478], ['imagine', 0.19701], ['in', 0.00106], ['into', 0.06015], ['involved_in', 0.08472], ['is', 0.00336], ['it', 0.02724], ['jeff_zeleny', 0.18756], ['left', 0.04269], ['man', 0.09986], ['manning', 0.28054], ['me', 0.04228], ['mediaite', 0.15237], ['might', 0.06118], ['most_popular', 0.04719], ['no', 0.03016]

In [15]:
"""
Function to see if any words in the title are present in the climate seed
"""
def calcTitleFactor(index):
  factor = 1
  for word in cleaned_titles[index].split():
    if word in climate_seed:
      factor = factor + 1
  
  return factor

def testCalcTitleFactor(sentence):
  factor = 1
  for word in sentence.split():
    print(word)
    if word in climate_seed:
      factor = factor + 1
  
  return factor

# Increasing the factor by 0.5 for every word in the climate seed that appears
# in the title of the article - will use that factor and multiply the tf-idf
# averaged score for the article by it to give it more precedence

In [16]:
"""
Function to calculate the averaged tf-idf for each article
"""
def calcXWeatherTFIDF():
    
    totalXWeatherScore = [0]*(len(cleaned_articles))
    index = 0
                
    for doc in tfidf[corpus]:
      num_words = 1
      total_score = 0
      for id, freq in doc:
        num_words = num_words + 1
        if mydict[id] in climate_seed:
          total_score = total_score + freq
      factor = calcTitleFactor(index)
      totalXWeatherScore[index] = (total_score/num_words)*factor # might be able to use idx here instead
      index = index + 1
                
    return totalXWeatherScore

In [17]:
"""
Calculating the averaged tf-idf for each article and storing
it as a list in result_of_xweather
"""
result_of_xweather = calcXWeatherTFIDF()

In [19]:
"""
Finding which articles have tf-idf averages greater than 0
meaning some of the seed words exist in the article
"""
indices_to_keep = []
for i in range(len(result_of_xweather)):
    if result_of_xweather[i] > 0:
        indices_to_keep.append(i)

print(len(indices_to_keep))

1117


In [20]:
new_list = [result_of_xweather[i] for i in indices_to_keep]

In [21]:
# list1, list2 = (list(t) for t in zip(*sorted(zip(list1, list2))))
new_list, indices_to_keep = (list(t) for t in zip(*sorted(zip(new_list, indices_to_keep), reverse=True)))
print("len(new_list) = ", len(new_list))
print("len(indices_to_keep) = ", len(indices_to_keep))
# new_list

len(new_list) =  1117
len(indices_to_keep) =  1117


In [22]:
"""
Function to divide the lists based on the threshold given
"""

def divide_threshold(article_list, index_list, top_percentage):
  range_end = int(len(article_list)*top_percentage)
  top_articles = article_list[:range_end]
  top_indices = index_list[:range_end]
  return top_articles, top_indices
  

In [23]:
 # Top 60% of articles
top_articles, top_indices = divide_threshold(new_list, indices_to_keep, 0.6)
len(top_articles)

670

In [27]:
final_breitbart = breitbart.iloc[top_indices]
final_breitbart.head()['content']

18427    MOST POPULAR[Sunday on CNN’s “Stare of the Uni...
29281    MOST POPULAR[Wednesday in San Francisco, durin...
38892    MONTPELIER, Vt. (AP) — Colleges and universiti...
25918    MOST POPULAR[Wednesday in San Francisco, durin...
28594    MOST POPULAR[CNN Correspondent Tom Foreman dec...
Name: content, dtype: object

In [28]:
final_breitbart.to_csv('../data/data_breitbart.csv')

In [29]:
for index, row in final_breitbart.iterrows():
    print(row['content'])


MOST POPULAR[Sunday on CNN’s “Stare of the Union.” during his interview with U.N. Ambassador Nikki Haley, host Jake Tapper called President Donald Trump’s past tweets on climate change “a big box of crazy.”, advertisement, Follow Pam Key On Twitter @pamkeyNEN 
MOST POPULAR[Wednesday in San Francisco, during his speech to the American Geophysical Union, Gov. Jerry Brown (D-CA) while vowing to the crowd of climate scientists that he will “fight” President-elect Donald Trump’s administration on climate change policy, Brown took a swipe at Breitbart News Network. advertisement, Brown said “The measures on methane that I signed into law, Breitbart and the other clowns talked about cow farts. That’s what they reduce. Everything is reduced from a serious to just it’s a joke. Well it’s not a joke.”, Follow Pam Key on Twitter @pamkeyNEN
MONTPELIER, Vt. (AP) — Colleges and universities worldwide are incorporating into their curriculums the evolving genre of literature that focuses on the changes